# 1. Getting VizDoom Up and Running

In [1]:
!pip install vizdoom

In [2]:
!cd github & git clone https://github.com/mwydmuch/ViZDoom

fatal: destination path 'ViZDoom' already exists and is not an empty directory.


In [3]:
# Import vizdoom for game env
from vizdoom import *
# Import random for action sampling
import random
# Import time for sleeping
import time
# Import numpy for identity matrix
import numpy as np

C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
# Setup game
game = DoomGame()
game.load_config('github/ViZDoom/scenarios/basic.cfg')
game.init()

In [5]:
actions = np.identity(3, dtype=np.uint8)

In [6]:
random.choice(actions)

array([0, 0, 1], dtype=uint8)

In [7]:
episodes = 10
for episode in range(episodes):
    game.new_episode()
    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        info = state.game_variables
        reward = game.make_action(random.choice(actions))
        print('reward :', reward)
        time.sleep(0.02)
    print('Result: ', game.get_total_reward())
    time.sleep(2)

reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : 100.0
reward : -1.0
Result:  94.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -6.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -6.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -6.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : 100.0
reward : -1.0
Result:  32.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : 100.0
reward : -1.0
Result:  95.0
reward : -1.0
rew

reward : -1.0
reward : -1.0
reward : -1.0
reward : -6.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -6.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -6.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -6.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward

reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -6.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -6.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -6.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward : -1.0
reward

In [8]:
game.close()

# 2. Converting it to a Gym Environment

In [9]:
!pip install gym

In [10]:
# Import environment base class from OpenAI Gym
from gym import Env
# Import gym spaces
from gym.spaces import Discrete, Box
# Import opencv
import cv2